In [7]:
!pip install scikit-image --upgrade
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Testing for single images using mutual infomation

In [8]:
#Mutual Information
import cv2
import numpy as np
from sklearn.metrics import mutual_info_score
import os

# Load query image
query_img = cv2.imread("/content/sample_data/data/dancing_10.jpg", cv2.IMREAD_GRAYSCALE)

# Load dataset images and extract SIFT features
sift = cv2.xfeatures2d.SIFT_create(nfeatures=1000)

# Set same SIFT parameters for query and dataset images
kp_query, des_query = sift.detectAndCompute(query_img, None)
dataset_features = []

# Load images from dataset folder and extract SIFT features
dataset_path = '/content/sample_data/data'
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename), cv2.IMREAD_GRAYSCALE)
    kp, des = sift.detectAndCompute(img, None)
    dataset_features.append(des)

# Stack dataset features into a NumPy array
dataset_features = np.vstack(dataset_features)

# Flatten query and dataset features
des_query = des_query.ravel()
dataset_features = dataset_features.ravel()

# Calculate mutual information score
print(len(des_query),len(dataset_features))
mi_score = mutual_info_score(des_query, dataset_features[:len(des_query)])

print("Mutual information score:", mi_score)


128000 3570048
Mutual information score: 0.15055666374991106


In [9]:
#Mutual Information
import cv2
from sklearn.metrics import mutual_info_score
import os

# Load the query image and convert to grayscale
query_img = cv2.imread('/content/sample_data/data/dancing_9.jpg')
query_img_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

# Convert the query image to a cv::UMat object
query_img_um = cv2.UMat(query_img_gray)

# Initialize SIFT feature extractor
sift = cv2.SIFT_create()

# Calculate mutual information between the query image and each image in the dataset folder
dataset_folder = '/content/sample_data/data'
mi_scores = []
for img_file in os.listdir(dataset_folder):
    if img_file.endswith('.jpg') or img_file.endswith('.png'):
        # Load the dataset image and convert to grayscale
        dataset_img = cv2.imread(os.path.join(dataset_folder, img_file))
        dataset_img_gray = cv2.cvtColor(dataset_img, cv2.COLOR_BGR2GRAY)

        # Convert the dataset image to a cv::UMat object
        dataset_img_um = cv2.UMat(dataset_img_gray)

        # Extract SIFT features from the query image and calculate mutual information
        kp_query, des_query = sift.detectAndCompute(query_img_um, None)
        kp_dataset, des_dataset = sift.detectAndCompute(dataset_img_um, None)
        des_query = des_query.get().ravel()
        dataset_features = des_dataset.get().ravel()
        # mi_score = mutual_info_score(des_query.get().ravel(), des_dataset.get().ravel())
        min_length = min(len(des_query),len(dataset_features))
        mi_score = mutual_info_score(des_query[:min_length], dataset_features[:min_length])
        mi_scores.append(mi_score)

# Rank the images in the dataset based on the mutual information scores and retrieve the top-k images
top_k = 5
top_k_indices = sorted(range(len(mi_scores)), key=lambda i: mi_scores[i], reverse=True)[:top_k]
top_k_images = [os.listdir(dataset_folder)[i] for i in top_k_indices]

# Print the top-k images
print("Top-k images:", top_k_images)


Top-k images: ['dancing_9.jpg', 'dancing_32.jpg', 'dancing_12.jpg', 'dancing_19.jpg', 'dancing_24.jpg']


# SSIM

In [10]:
# #SSIM
# from skimage import io, img_as_float
# from skimage.metrics import structural_similarity as ssim
# from skimage.transform import resize

# # Load the reference and comparison images
# image_ref = img_as_float(io.imread('/content/sample_data/data/dancing_11.jpg'))
# image_comp = img_as_float(io.imread('/content/sample_data/data/dancing_12.jpg'))

# image_ref = resize(image_ref, image_comp.shape, anti_aliasing=True)
# image_comp = resize(image_comp, image_ref.shape, anti_aliasing=True)

# # Calculate the SSIM between the two images
# ssim_index = ssim(image_ref, image_comp, win_size=3, multichannel=True)

# print("SSIM index:", ssim_index)


In [11]:
#SSIM
import os
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim

# Define the query image and its dimensions
query_image = 'query_image.jpg'
height, width = 256, 256

# Define the query image and its dimensions
query_image = '/content/sample_data/data/dancing_10.jpg'
# Define the dataset directory
dataset_dir = '/content/sample_data/data'

# Initialize an empty list to store the SSIM indices for each image in the dataset
ssim_indices = []

# Load the query image and resize it to the desired dimensions
query_image = Image.open(query_image).resize((height, width)).convert('RGB')

# Loop over all images in the dataset directory and calculate their SSIM index with the query image
for filename in os.listdir(dataset_dir):
    # Load the image from file and resize it to the same size as the query image
    image = Image.open(os.path.join(dataset_dir, filename)).resize((height, width)).convert('RGB')
    # Calculate the SSIM index between the query image and the current image in the dataset
    ssim_index = ssim(np.array(query_image), np.array(image), win_size=3, multichannel=True)
    # Append the SSIM index and the image filename to the ssim_indices list
    ssim_indices.append((ssim_index, filename))
print(ssim_indices[:1])
# Sort the ssim_indices list in descending order based on the SSIM index and select the top 5 images
top_5_images = sorted(ssim_indices, reverse=True)[:5]

# Print the filenames of the top 5 images and their corresponding SSIM indices
for ssim_index, filename in top_5_images:
    print(f'{filename}: {ssim_index:.4f}')


<ipython-input-11-36ac92236d72>:27: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_index = ssim(np.array(query_image), np.array(image), win_size=3, multichannel=True)


[(0.34872852964381185, 'dancing_21.jpg')]
dancing_10.jpg: 1.0000
dancing_17.jpg: 0.3670
dancing_2.jpg: 0.3617
dancing_21.jpg: 0.3487
dancing_15.jpg: 0.3005
